In [ ]:
##Week 10 - Building off of Week 9 (getting rid of uneven genres)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
df_test_encoded = pd.read_csv(r'week_5_test_encoded.csv')

df_train_encoded = pd.read_csv(r'week_5_train_encoded.csv')

df_val_encoded = pd.read_csv(r'week_5_validation_encoded.csv')

In [ ]:
combined_df = pd.concat([df_train_encoded, df_test_encoded, df_val_encoded], ignore_index=True)

In [ ]:
combined_df.columns

Index(['Song', 'Artist', 'Popularity', 'BPM', 'Dance', 'Energy', 'Acoustic',
       'Happy', 'Loud', 'Clean_Lyrics', 'Power', 'sqrt_Acoustic', 'sqrt_BPM',
       'camelot_sin', 'camelot_cos', 'Genre_alternative', 'Genre_country',
       'Genre_dance', 'Genre_electronic', 'Genre_folk', 'Genre_hip hop',
       'Genre_indie', 'Genre_pop', 'Genre_rnb', 'Genre_rock', 'Genre_soul',
       'Subgenre_alternative', 'Subgenre_alternative rnb',
       'Subgenre_alternative rock', 'Subgenre_country', 'Subgenre_country pop',
       'Subgenre_dance', 'Subgenre_dance electronic', 'Subgenre_dance pop',
       'Subgenre_electronic', 'Subgenre_electronic dance',
       'Subgenre_electronic pop', 'Subgenre_folk', 'Subgenre_folk rock',
       'Subgenre_hip hop', 'Subgenre_hip hop 90s', 'Subgenre_hip hop rnb',
       'Subgenre_indie', 'Subgenre_indie alternative', 'Subgenre_indie folk',
       'Subgenre_indie pop', 'Subgenre_indie rock', 'Subgenre_pop',
       'Subgenre_pop 60s', 'Subgenre_pop 70s', 'Subge

In [ ]:
print(combined_df.select_dtypes(include=['number']).columns)


Index(['Popularity', 'BPM', 'Dance', 'Energy', 'Acoustic', 'Happy', 'Loud',
       'Power', 'sqrt_Acoustic', 'sqrt_BPM', 'camelot_sin', 'camelot_cos',
       'Genre_alternative', 'Genre_country', 'Genre_dance', 'Genre_electronic',
       'Genre_folk', 'Genre_hip hop', 'Genre_indie', 'Genre_pop', 'Genre_rnb',
       'Genre_rock', 'Genre_soul', 'Subgenre_alternative',
       'Subgenre_alternative rnb', 'Subgenre_alternative rock',
       'Subgenre_country', 'Subgenre_country pop', 'Subgenre_dance',
       'Subgenre_dance electronic', 'Subgenre_dance pop',
       'Subgenre_electronic', 'Subgenre_electronic dance',
       'Subgenre_electronic pop', 'Subgenre_folk', 'Subgenre_folk rock',
       'Subgenre_hip hop', 'Subgenre_hip hop 90s', 'Subgenre_hip hop rnb',
       'Subgenre_indie', 'Subgenre_indie alternative', 'Subgenre_indie folk',
       'Subgenre_indie pop', 'Subgenre_indie rock', 'Subgenre_pop',
       'Subgenre_pop 60s', 'Subgenre_pop 70s', 'Subgenre_pop 80s',
       'Subgenre_pop

Looking at the numerical columns for our dataset we see that most of them come from the genre one-hot encoding. For now we will continue to omit these columns from the model.

Last week's model that incorporated the musical numerical features performed slightly better than the model's using just embedding. However, it was noticed that the best model incorporated BPM, sqrt_BPM, Acoustic, and aqrt_Acoustic. This is not good becuase those metrics were double weighted in the model. This week, part of our updated model is addressing that which we will do when creating the model.

Our model will continue to use the Glove-Twitter (Word2Vec) embeddings as they provided a better accuracy score compared to the original embedding method.


In [ ]:
#Using word2vec this week (from week 7 notebook)

import gensim.downloader as api
model = api.load("glove-twitter-200")

In [ ]:
#Function that converts a list of tokenized words into a single (fixed length) vector using the glove-twitter-200 model
def get_embedding_twitter(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [ ]:
#Creates a column called 'Twitter_Embeddings" in all three of the dataframes/datasets by applying the get_embedding_twitter() function to the tokens list

combined_df['Twitter_Embeddings'] = combined_df['Tokens'].apply(lambda tokens: get_embedding_twitter(tokens, model))

In [ ]:
#Allows us to work with the vector numerically
def str_to_embedding(s):
    # Remove [ and ] characters
    s = s.strip('[]')
    # Split by spaces and convert each to float
    return np.array([float(x) for x in s.split()])

# Example usage:
embedding_str = "[-5.49920015e-02 -4.24173214e-02 1.20421372e-02]"
embedding_vector = str_to_embedding(embedding_str)
print(embedding_vector)

[-0.054992   -0.04241732  0.01204214]


In [ ]:
#Converts the string in 'Embeddings' column into numpy array using the above function
combined_df['Embeddings'] = combined_df['Embeddings'].apply(str_to_embedding)

In [ ]:
combined_df.head()

,Song,Artist,Popularity,BPM,Dance,Energy,Acoustic,Happy,Loud,Clean_Lyrics,...,Subgenre_rock alternative,Subgenre_soul,Subgenre_soul 60s,Subgenre_soul 70s,Subgenre_soul 80s,Subgenre_soul disco,Subgenre_soul rnb,Tokens,Embeddings,Twitter_Embeddings
0,Stretch You Out (feat. A Boogie wit da Hoodie),"Summer Walker,A Boogie Wit da Hoodie",0.50,0.317647,0.623529,0.494737,0.252525,0.231579,0.76,get london da track niggas insecure claim enou...,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"['get', 'london', 'da', 'track', 'niggas', 'in...","[-0.0549920015, -0.0424173214, 0.120421372, -0...","[0.073948905, -0.1129652, -0.046067417, 0.0734..."
1,On Melancholy Hill,Gorillaz,0.80,0.417647,0.682353,0.726316,0.000000,0.578947,0.80,melancholy hill good side consumerism like muc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['melancholy', 'hill', 'good', 'side', 'consum...","[-0.0140985716, -0.0445414186, 0.111462869, 0....","[0.10808624, -0.1391475, -0.055444743, 0.04700..."
2,WHO CARES?,Rex Orange County,0.35,0.217647,0.835294,0.242105,0.595960,0.589474,0.80,mmmm mmmm mmmm mmmm first time try free doubt ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['mmmm', 'mmmm', 'mmmm', 'mmmm', 'first', 'tim...","[-0.00185818132, -0.0364763662, 0.147965446, 0...","[0.07740225, -0.12858482, -0.05046654, 0.07772..."
3,Solid,Ashford & Simpson,0.45,0.305882,0.823529,0.442105,0.272727,0.978947,0.56,love sake mistake oh forgave soon learn trust ...,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"['love', 'sake', 'mistake', 'oh', 'forgave', '...","[-0.0136662908, -0.0114702322, 0.119016811, 0....","[0.053561937, -0.07968545, -0.08123524, 0.0626..."
4,BREAK MY SOUL,Beyoncé,0.55,0.388235,0.682353,0.884211,0.060606,0.863158,0.84,'bout explode take load bend bust open ya make...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[""'bout"", 'explode', 'take', 'load', 'bend', '...","[-0.0563979559, -0.0503409989, 0.103356943, -0...","[0.06484171, -0.08521436, -0.048548214, 0.1225..."


In [ ]:
print(type(combined_df['Embeddings'].iloc[0]))
print(type(combined_df['Twitter_Embeddings'].iloc[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
#Converts the "Embeddings" column into a 2D numpy array (matrix) where each row represents a song
X = np.stack(combined_df['Embeddings'].values)

Fitting the Week 9 KNN model
1. Excluding 'Acoustic' and 'BPM' as the sqrt transformations of those metrics give a more normal distrubution
2. Weighing lyric embeddings and music features the same -- last week embeddings were weighed equal to any other metric such as BPM. We think lyrics are a significant part of music (half) and therefore in this model the lyric embeddings will be weighed 50% of the total distance and the remaining numerical metadata will collectively weigh the other 50% of the distance.
3. Now including genres slightly - hopefully to get country songs better aligned

Updated Model - With re evaluation of genres included, including all columns
1. Removing genres that are outliers (low amounts of songs in sample)
2. Adding back in Artist in order to recommend similar songs by the same artist
3. Adding in Subgenre to see if this helps our model as well

In [ ]:
# Get all genre columns (start with 'Genre_')
genre_cols = [col for col in combined_df.columns if col.startswith('Genre_')]

# Sum values in each genre column (assuming 1 means the song belongs to that genre)
genre_counts = combined_df[genre_cols].sum().sort_values(ascending=False)

print("Genre Counts:")
print(genre_counts)


Genre Counts:
Genre_pop            546.0
Genre_indie          539.0
Genre_rock           539.0
Genre_hip hop        327.0
Genre_alternative    273.0
Genre_country        193.0
Genre_soul           152.0
Genre_electronic     143.0
Genre_folk           112.0
Genre_dance          111.0
Genre_rnb             86.0
dtype: float64


From these values we decided to remove the genres folk (112 songs), dance (111 songs), and rnb (86 songs) from the dataset due to their relatively low representation compared to dominant genres like pop, indie, and rock, which have counts around 540. Including these lower-frequency genres could introduce noise and imbalance in the model, potentially leading to overfitting on small sample sizes and weaker generalization. Removing them helps focus the model on genres with sufficient data to learn meaningful patterns, improving robustness and predictive accuracy.



In [ ]:
# Filter condition - keeping rows where none of the genres dance, folk, or rnb are 1
filtered_df = combined_df[
    (combined_df['Genre_dance'] != 1) &
    (combined_df['Genre_folk'] != 1) &
    (combined_df['Genre_rnb'] != 1)
].copy()

In [ ]:
#Removing the song, clean lyrics, tokens, embeddings, twitter embeddings, bpm and acoustic still

columns_to_exclude = [col for col in filtered_df.columns
                      if col in ['Song', 'Clean_Lyrics', 'Tokens', 'Embeddings',
                              'Twitter_Embeddings', 'BPM', 'Acoustic']]


In [ ]:
# Get numeric metadata features - numerical data with only the columns we want (no embeddings)
X_meta = filtered_df.drop(columns=columns_to_exclude).select_dtypes(include=[np.number])

# Extract Twitter embeddings
X_embed = np.stack(filtered_df['Twitter_Embeddings'].values)

# Extract genre features (one-hot encoded)
X_genre = filtered_df.filter(like='Genre_')

# Apply weighting so that each block contributes 50% to the distance
m = X_meta.shape[1]   # number of metadata features
g = X_genre.shape[1]
e = X_embed.shape[1]  # number of embedding dimensions

# Weighting: ensure total sums to 1.0
meta_weight = np.sqrt(0.45 / m)
genre_weight = np.sqrt(0.1 / g)
embed_weight = np.sqrt(0.45 / e)

# Weighted feature arrays
X_full = np.hstack([
    X_meta * meta_weight,
    X_genre * genre_weight,
    X_embed * embed_weight
])


In [ ]:
# Fit the NEW KNN_Weighted model with the data updates
KNN_Weighted = NearestNeighbors(n_neighbors=6, metric='cosine')
KNN_Weighted.fit(X_full)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [ ]:
# Step: Example recommendation for one song from the combined dataset with data changes
test_song_idx = 100  # Index of the song to base recommendations on
distances, indices = KNN_Weighted.kneighbors([X_full[test_song_idx]])

print(f"\nRecommendations for song: {filtered_df['Song'].iloc[test_song_idx]} by {filtered_df['Artist'].iloc[test_song_idx]}")

# Skip the first match (which would be the song itself)
for rank, (idx, dist) in enumerate(zip(indices[0][1:], distances[0][1:]), start=1):
    song = filtered_df['Song'].iloc[idx]
    artist = filtered_df['Artist'].iloc[idx]
    print(f"{rank}. {song} by {artist} (distance: {dist:.4f})")



Recommendations for song: Dancing in the Moonlight by Toploader
1. Miles On It by Marshmello,Kane Brown (distance: 0.0062)
2. Too Sweet by Hozier (distance: 0.0063)
3. I'm Like A Bird by Nelly Furtado (distance: 0.0090)
4. Smile by Lily Allen (distance: 0.0114)
5. Leave The Door Open by Bruno Mars,Anderson .Paak,Silk Sonic (distance: 0.0118)


In [ ]:
# Get top 2 neighbors for each song (so we can skip the self-match)
full_distances, full_indices = KNN_Weighted.kneighbors(X_full, n_neighbors=2)

full_recommendations_week10 = []

# Generate recommendations for the first 100 songs
for i in range(100):
    test_song_title = filtered_df.iloc[i]['Song']
    test_artist = filtered_df.iloc[i]['Artist']

    rec_idx = full_indices[i][1]  # [1] skips the self-match at [0]
    rec_song_title = filtered_df.iloc[rec_idx]['Song']
    rec_artist = filtered_df.iloc[rec_idx]['Artist']

    full_recommendations_week10.append({
        'Test Song Title': test_song_title,
        'Test Artist': test_artist,
        'Recommended Song Title': rec_song_title,
        'Recommended Artist': rec_artist
    })

# Save to CSV
full_rec_df_week10 = pd.DataFrame(full_recommendations_week10)
full_rec_df_week10.to_csv('top_100_knn_weighted_w_genre_recommendations_week10.csv', index=False)

print("Saved top 100 KNN_Weighted recommendations to 'top_100_knn_weighted_w_genre_recommendations_week10.csv'")


Saved top 100 KNN_Weighted recommendations to 'top_100_knn_weighted_w_genre_recommendations_week10.csv'
